In [ ]:
import pipline.Extractor as E
import pipline.Transformer as T
import pipline.Loader as L

In [81]:
import time
import Scrapping as scrp
import bs4 as bs
from selenium.webdriver.common.by import By

# It opens a browser, scrolls down to the bottom of the page, and then returns the html code of the
# page

class extractor:
    
    def __init__(self, url, nb_tracks=None, date_cible=None):
        self.url = url
        self.nb_tracks = nb_tracks
        self.date_cible = date_cible
        
    def extracting_tag(self):
        return self.url.split("/")[-1]
    
    def extract(self):
            
            def fixdate(str):
                jour = str.split("T")[0]
                heure = str.split("T")[1].split(".")[0]
                datesql = f'{jour} {heure}'
                return datesql

            print("<<DEBUT EXTRACTION>>")

            print("Ouverture de Chrome...")
            browser = scrp.Scrapping(self.url).html_scrapping()

            #Appuiyer sur "accepter les cookies"
            print("Clic sur accepter cookies...")
            button_cookie_agree = browser.find_element(By.ID, 'onetrust-accept-btn-handler')
            button_cookie_agree.click()
            time.sleep(2)

            if self.date_cible:

                print("Première tracks trouvé !")
                data_item = bs.BeautifulSoup(browser.page_source, "html.parser")
                item_datetime = fixdate(data_item.find("time", {"class": "relativeTime sc-text-secondary sc-text-captions"}).get('datetime'))
                print(f"Date du premier item : {item_datetime}")

                while item_datetime>=self.date_cible:

                    browser.execute_script("window.scrollTo(0, 100000000)")
                    time.sleep(2)
                    data_item = bs.BeautifulSoup(browser.page_source, "html.parser")
                    list_data_item=data_item.find_all("time", {"class": "relativeTime sc-text-secondary sc-text-captions"})
                    item_datetime = fixdate(list_data_item[-1].get('datetime'))
                    
                    print(f"Date de l'item : {item_datetime}")
                    print("Scroll/")
            else:

                print("Première tracks trouvé !")
                item = browser.count_elements(By.CLASS_NAME, "soundList_item")

                #Scroll down pour faire apparaitre plus d'item SC
                while item<=self.nb_tracks:

                    browser.execute_script("window.scrollTo(0, 10000)")
                    time.sleep(2)
                    item = browser.count_elements(By.CLASS_NAME, "soundList_item")
                    print("Scroll/")

            print(f"Nombre de tracks repéré : {len(list_data_item)}")

            print("Extraction du code source de la page...")
            data=data_item

            print("Fermeture de Chrome...")
            browser.quit() 

            print("<<FIN EXTRACTION>>")

            return data

In [82]:
URL="https://soundcloud.com/tags/acidcore"
ITEM_NB = 9

extractor = extractor(URL,None, "2023-03-24 00:00:00")
extracted_data = extractor.extract()

<<DEBUT EXTRACTION>>
Ouverture de Chrome...


KeyboardInterrupt: 

In [3]:
# It's a class that transforms data

class transformer:
    
    def __init__(self):
        pass
            
    def fixstr(self):
        return self.replace("'", "''")

    def fixdate(self):
        jour = self.split("T")[0]
        heure = self.split("T")[1].split(".")[0]
        return f'{jour} {heure}'

    def fixinteger(self):
        try:
            strlist = self.split(".")
            new_int = int(''.join(strlist))
        except ValueError:
            strlist = self.split(",")
            new_int = int(''.join(strlist))
        return new_int

    def transform(data):
        # sourcery skip: extract-duplicate-method, instance-method-first-arg-name
        print("")
        print("<<DEBUT TRANSFORMATION>>")

        print("Récupération des items contnenant les données...")
        tracks = data.find_all("li", {"class": "soundList__item"})

        print("Transformation des données... ")
        print("")
        datas = []
        for track in tracks:

            d_data = {
                "artist": transformer.fixstr(track.find("span", {"class": "soundTitle__usernameText"}).get_text(strip=True)),
                "date": transformer.fixdate(track.find("time", {"class": "relativeTime sc-text-secondary sc-text-captions"}).get('datetime')),
                }
            try:
                d_data["title"] = transformer.fixstr(track.find("a", {"class": "sc-link-primary soundTitle__title sc-link-dark sc-text-h4"}).findChildren()[0].get_text(strip=True))
            except(AttributeError):
                d_data["title"] = transformer.fixstr(track.find("a", {"class": "sc-link-primary soundTitle__title g-opacity-transition-500 g-type-shrinkwrap-block g-type-shrinkwrap-primary theme-dark sc-text-h4"}).findChildren()[0].get_text(strip=True))

            try:
                d_data["num_likes"] = transformer.fixinteger(track.find("button", {"class": "sc-button-like sc-button-secondary sc-button sc-button-small sc-button-responsive"}).get_text())
            except(ValueError):
                d_data["num_likes"] = 0

            if track.find("a", {"class": "sc-ministats sc-ministats-small sc-ministats-comments sc-link-secondary"}):
                d_data["num_comments"] = transformer.fixinteger(track.find("a", {"class": "sc-ministats sc-ministats-small sc-ministats-comments sc-link-secondary"}).findChildren()[1].get_text())
            else:
                d_data["num_comments"] = 0

            if track.find("span", {"class": "sc-ministats sc-ministats-small sc-ministats-plays sc-text-secondary"}):
                d_data["num_streams"] = transformer.fixinteger(track.find("span", {"class": "sc-ministats sc-ministats-small sc-ministats-plays sc-text-secondary"}).findChildren()[1].get_text())
            else:
                d_data["num_streams"] = 0

            if track.find("span", {"class": "sc-truncate sc-tagContent"}):
                if transformer.fixstr(track.find("span", {"class": "sc-truncate sc-tagContent"}).get_text())==data.find("h1", {"class": "tagsMain__title sc-py-1x sc-px-2x sc-mb-2x"}).get_text().split("#")[-1]:
                    d_data["main_tag"] = transformer.fixstr(track.find("span", {"class": "sc-truncate sc-tagContent"}).get_text())
                else:
                    d_data["main_tag"] = transformer.fixstr(track.find("span", {"class": "sc-truncate sc-tagContent"}).get_text())
                    d_data["list_tags"] = data.find("h1", {"class": "tagsMain__title sc-py-1x sc-px-2x sc-mb-2x"}).get_text().split("#")[-1]
            else:
                d_data["main_tag"] = "Null"

            if track.find("a", {"class": "sc-ministats sc-ministats-small sc-link-secondary sc-text-h4 sc-ministats-followers "}):
                d_data["num_abonnes"] = transformer.fixinteger(track.find("a", {"class": "sc-ministats sc-ministats-small sc-link-secondary sc-text-h4 sc-ministats-followers "}).findChildren()[1].get_text())
            else:
                d_data["num_abonnes"] = 0

            print("Nouvelle ligne :")
            print(d_data)
            print("")

            """
            print("titre : " + str(d_data["title"]) + "\n" +
            "artist : " + str(d_data["artist"]) + "\n" +
            "date : " + str(d_data["date"]) + "\n" +
            "num_likes : " + str(d_data["num_likes"]) + "\n" +
            "num_comments : " + str(d_data["num_comments"]) + "\n" +
            "num_streams : " + str(d_data["num_streams"]) + "\n" +
            "main_tag : " + str(d_data["main_tag"]))
            """

            datas.append(d_data)

        print("<<FIN EXTRACTION>>")

        return datas

In [4]:
transformed_data = transformer.transform(extracted_data)


<<DEBUT TRANSFORMATION>>
Récupération des items contnenant les données...
Transformation des données... 

Nouvelle ligne :
{'artist': 'Arthur_cblln', 'date': '2023-03-23 18:07:11', 'title': 'Mix Acidcore', 'num_likes': 0, 'num_comments': 0, 'num_streams': 0, 'main_tag': 'Acidcore', 'list_tags': 'acidcore', 'num_abonnes': 0}

Nouvelle ligne :
{'artist': 'Lolo', 'date': '2023-03-23 17:44:34', 'title': 'The world is burning - Tekno Acid', 'num_likes': 2, 'num_comments': 0, 'num_streams': 7, 'main_tag': 'Tekno Acid', 'list_tags': 'acidcore', 'num_abonnes': 0}

Nouvelle ligne :
{'artist': 'Nemesis', 'date': '2023-03-23 17:13:07', 'title': 'La Danse Du Serpent', 'num_likes': 2, 'num_comments': 1, 'num_streams': 8, 'main_tag': 'Tekno', 'list_tags': 'acidcore', 'num_abonnes': 0}

Nouvelle ligne :
{'artist': 'Chroma.wav', 'date': '2023-03-23 15:17:17', 'title': 'La Boucle Des Narkotik', 'num_likes': 7, 'num_comments': 4, 'num_streams': 22, 'main_tag': 'acidcore', 'num_abonnes': 0}

Nouvelle li

In [5]:
import Connect as co

# It takes a list of dictionaries and inserts the data into a PostgreSQL database

class loader:
    
    def __init__(self,):
        pass
    
    def load(self) -> None:  # sourcery skip: or-if-exp-identity
        print("")
        print("<<DEBUT LOADING>>")

        conn = co.connect.connection()
        print('Connexion à la base de données établie !')

        cur = conn.cursor()

        print("DEBUT REQUÊTES INSERTION")
        for i in range(len(self)):
            
            try:
                #Requête d'insertion table artiste
                sql_artists = (
                    "INSERT INTO public.artists(artist_name, num_sub) VALUES ('%s','%d') ON CONFLICT DO NOTHING;"
                    % (self[i]["artist"], self[i]["num_abonnes"])
                )
                cur.execute(sql_artists)
                print("")
                print("Requête %d artiste : %s" %(i,sql_artists))

            except UnicodeEncodeError as error:
                
                print(f"Probleme enconding data dans la requête : {error.args[1]}")
                #Si un problème d'encodage et raise alors la valeur qui pose probleme sera remplacé par Null

                self[i]["artist"] = "Null"
                sql_artists = (
                    "INSERT INTO public.artists(artist_name, num_sub) VALUES ('%s',%d) ON CONFLICT DO NOTHING;"
                    % (self[i]["artist"], self[i]["num_abonnes"])
                )
                cur.execute(sql_artists)
                print("Requête %d artiste avec null : %s" %(i,sql_artists))

            for key in range(len(self[i])-7):
                try:    
                    #Requête d'insertion table tags
                    sql_tags = f"""INSERT INTO public.tags(tag_name) VALUES ('{self[i]["main_tag"] if key == 0 else self[i]["taglist"]}') ON CONFLICT DO NOTHING;"""
                    cur.execute(sql_tags)
                    print("Requête %d tag : %s" %(i,sql_tags))

                except UnicodeEncodeError as error:
                    
                    print(f"Probleme enconding data dans la requête : {error.args[1]}")
                    #Si un problème d'encodage et raise alors la valeur qui pose probleme sera remplacé par Null

                    if key == 0:
                        self[i]["main_tag"] = "Null"  
                    else:
                        self[i]["taglist"] = "Null"
                    
                    sql_tags = f"""INSERT INTO public.tags(tag_name) VALUES ('{self[i]["main_tag"] if key == 0 else self[i]["taglist"]}') ON CONFLICT DO NOTHING;"""
                    cur.execute(sql_tags)
                    print("Requête %d tag avec Null : %s" %(i,sql_tags))

            try:        
                #Requête d'insertion table tracks
                sql_tracks = (
                    """INSERT INTO public.tracks(
                        title, 
                        artists_id,
                        datep,
                        num_likes,
                        num_comments,
                        num_streams,
                        maintag
                        )
                        VALUES (
                            '%s',
                            (SELECT artists_id FROM artists WHERE artist_name = '%s'),
                            '%s',
                            '%d',
                            '%d',
                            '%d',
                            (SELECT tags_id FROM tags WHERE tag_name = '%s'),
                            ARRAY(SELECT tags_id FROM tags WHERE tag_name ='%s')
                        ) 
                        ON CONFLICT (title) DO 
                        UPDATE SET num_likes = %d, num_comments = %d, num_streams = %d WHERE tracks.title = '%s';"""
                    % (
                        self[i]["title"],
                        self[i]["artist"],
                        self[i]["date"],
                        self[i]["num_likes"],
                        self[i]["num_comments"],
                        self[i]["num_streams"],
                        self[i]["main_tag"],
                        self[i]["taglist"] if self[i]["taglist"] else "EXEPT",
                        self[i]["num_likes"],
                        self[i]["num_comments"],
                        self[i]["num_streams"],
                        self[i]["title"],
                    )
                )
                cur.execute(sql_tracks)
                print("Requête %d track : %s" %(i,sql_tracks))

            except UnicodeEncodeError as error:
                
                print(f"Probleme enconding data dans la requête : {error.args[1]}")
                #Si un problème d'encodage et raise alors la valeur qui pose probleme sera remplacé par Null

                self[i]["title"] = "Null"
                sql_tracks = (
                    "INSERT INTO public.tracks(title, artists_id, datep, num_likes, num_comments, num_streams, maintag) VALUES ('%s',(SELECT artists_id FROM artists WHERE artist_name = '%s'),'%s','%d','%d','%d',(SELECT tags_id FROM tags WHERE tag_name = '%s')) ON CONFLICT DO NOTHING;"
                    % (
                        'Null',
                        self[i]["artist"],
                        self[i]["date"],
                        self[i]["num_likes"],
                        self[i]["num_comments"],
                        self[i]["num_streams"],
                        self[i]["main_tag"],
                    )
                )
                cur.execute(sql_tracks)
                print("Requête %d track avec null : %s" %(i,sql_tracks))

            conn.commit()
            print("")
            print("Commit éffectué pour les 3 dernières requêtes !")

        print("")
        print("Fin des requête d'insertion")

        cur.close()
        print("Curseur fermé")
        conn.close()
        print("Connexion avec la base de données fermée")
        print("<<FIN LOADING>>")

In [13]:
def fixinteger(self):
        try:
            strlist = self.split(".")
            if self.find("K") != -1:
                new_int = int(strlist[0])*1000 + int(strlist[1].split("K")[0])*100
            elif self.find("M") != -1:
                new_int = int(strlist[0])*1000000 + int(strlist[1].split("M")[0])*100000
            else:
                new_int = int(''.join(strlist))
        except ValueError:
            strlist = self.split(",")
            if self.find("K") != -1:
                new_int = int(strlist[0])*1000 + int(strlist[1].split("K")[0])*100
            elif self.find("M") != -1:
                new_int = int(strlist[0])*1000000 + int(strlist[1].split("M")[0])*100000
            else:
                new_int = int(''.join(strlist))
        return new_int

In [83]:
def insert(table, cols, do = None):
    
    cols_key_list = [k  for (k, val) in cols.items()]
    if do : do_key_list = [k  for (k, val) in do.items()]

    attribs = cols_key_list[0]
    values = f"{cols[cols_key_list[0]]}"

    for i in range(1,len(cols_key_list)):
        
        attribs = f"{attribs},{cols_key_list[i]}"
        values = f"{values},{cols[cols_key_list[i]]}"

    do = f"ON CONFLICT ({do_key_list[0]}) DO {do[do_key_list[0]]}" if do else "ON CONFLICT DO NOTHING"

    return f"INSERT INTO {table}({attribs}) VALUES ({values}) {do};"
    
def select(cols, table, where):
    # sourcery skip: instance-method-first-arg-name

    if cols == "*":
        if where:
            sql = f"SELECT * FROM {table} WHERE {where}"
        else:
            sql = f"SELECT * FROM {table}"
    else:
        attribs = cols[0]
        for i in range(1,len(cols)):
            attribs = f"{attribs}, {cols[i]}"
        if where:
            sql = f"SELECT {attribs} FROM {table} WHERE {where}"
        else:
            sql = f"SELECT {attribs} FROM {table}"
            
    return sql

def updateSet(cols, where):
    key_list = [k  for (k, val) in cols.items()]
    set_list = f"{key_list[0]} = {cols[key_list[0]]}"
    for i in range(1,len(key_list)):
        set_list = f"{set_list}, {key_list[i]} = {cols[key_list[i]]}"
    
    return f"UPDATE SET {set_list} WHERE {where}"

In [85]:
data = ({'artist': 'TekeT', 'date': '2023-03-27 19:00:03', 'title': 'TekeT - Machines Education Part 1 (Live session)', 'num_likes': 1, 'num_comments': 0, 'num_streams': 6, 'main_tag': 'Mental/Acidcore', 'taglist': 'acidcore', 'num_abonnes': 0},{'artist': 'Nemesis', 'date': '2023-03-23 17:13:07', 'title': 'La Danse Du Serpent', 'num_likes': 2, 'num_comments': 1, 'num_streams': 8, 'main_tag': 'Tekno', 'taglist': 'acidcore', 'num_abonnes': 0})
for i in range (len(data)):
    print(insert("public.artists", {"artist_name": data[i]["artist"], "num_sub": data[i]["num_abonnes"]}))

print(
    f"""{select("*", "public.artists", f"artist_name = '{data[i]['artist']}'")};"""
)
print(select(["artist_name"],"public.artists", f"artist_name = '{data[i]['artist']}';"))
print(insert("public.tracks", 
                    {
                        "title": data[i]["title"], 
                        "artists_id": select(["artists_id"],"public_artist",f"artist_name = '{data[i]['artist']}'"),
                        "datep": data[i]["date"],
                        "num_likes": data[i]["num_likes"],
                        "num_comments": data[i]["num_comments"],
                        "num_streams": data[i]["num_streams"],
                        "tags_id": select(["tags_id"],"public.tags", f"tag_name = '{data[i]['main_tag']}'"),
                        "taglist": f"""ARRAY({data[i]["taglist"] if len(data[i]) != 8 else "Null"})"""
                    },
                    {"title" : updateSet(
                        {
                            "num_likes": f"'{data[0]['num_likes']}'", 
                            "num_comments": f"'{data[0]['num_comments']}'",
                            "num_streams": f"'{data[0]['num_streams']}'", 
                            "taglist": f"""ARRAY({select(["taglist"], 
                                                "public.tracks", 
                                                f"title = '{data[0]['title']}'")}) || ARRAY({select(["tags_id"], "public.tags", f"tag_name = '{data[i]['taglist'] if len(data[i]) != 8 else 'Null'}'")})"""
                        },
                        f"tracks.title = '{data[i]['title']}'"
                    )}
    ))


INSERT INTO public.artists(artist_name,num_sub) VALUES (TekeT,0) ON CONFLICT DO NOTHING;
INSERT INTO public.artists(artist_name,num_sub) VALUES (Nemesis,0) ON CONFLICT DO NOTHING;
SELECT * FROM public.artists WHERE artist_name = 'Nemesis';
SELECT artist_name FROM public.artists WHERE artist_name = 'Nemesis';
INSERT INTO public.tracks(title,artists_id,datep,num_likes,num_comments,num_streams,tags_id,taglist) VALUES (La Danse Du Serpent,SELECT artists_id FROM public_artist WHERE artist_name = 'Nemesis',2023-03-23 17:13:07,2,1,8,SELECT tags_id FROM public.tags WHERE tag_name = 'Tekno',acidcore) ON CONFLICT (title) DO UPDATE SET num_likes = '1', num_comments = '0', num_streams = '6', taglist = ARRAY(SELECT taglist FROM public.tracks WHERE title = 'TekeT - Machines Education Part 1 (Live session)') || ARRAY(SELECT tags_id FROM public.tags WHERE tag_name = 'acidcore') WHERE tracks.title = 'La Danse Du Serpent';


In [4]:
a = "artist_id"

if a in ("deux", "un"):
    print ("a est dans la liste")
else:
    print("a n'est pas dans a liste")

a est dans la liste


In [6]:
L.loader.load(transformed_data)


<<DEBUT LOADING>>
erreur : Section postgresql not found in the Python_Projects\SCDashBoard\database.ini file
Connexion à la base de données établie !


AttributeError: 'NoneType' object has no attribute 'cursor'